In [ ]:
file_path = 'data/train.txt'

with open(file_path, 'r') as f:
    first_line = f.readline()
    print(first_line.strip())

0,The boy went to a video arcade | He played his favorite machine | His games didn't go very well | He told the owner about his experience | The owner explained that he had made the game settings harder,He told the owner about his experience | He played his favorite machine | His games didn't go very well | The owner explained that he had made the game settings harder | The boy went to a video arcade


In [27]:
from roc_functions import preprocessing, randomize_story

preprocessing('data/train.txt', 'data/test.txt')

In [14]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv
import random

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

rng = random.Random(42)

with open("data/processed/train_processed.csv", 'r') as f:
    datareader = csv.reader(f)
    for row in datareader:
        original_story = row[1:2]
        shuffled_story = row[2]
    print("Original story:", " ".join(original_story))
    print("Shuffled story:", shuffled_story)

response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "user", "content": f"Reorder these sentences chronologically keeping the same format: {shuffled_story}"}
    ]
)
print(f"Reconstructed story: {response.choices[0].message.content}")

Original story: Starting my first day of college was terrifying | I was not sure of my professors or the people in the class | Once class started, my fears dissipated | The class was enjoyable and the other students were friendly | I now love school and I do not get nervous anymore
Shuffled story: Starting my first day of college was terrifying | Once class started, my fears dissipated | I now love school and I do not get nervous anymore | I was not sure of my professors or the people in the class | The class was enjoyable and the other students were friendly
Reconstructed story: Starting my first day of college was terrifying | I was not sure of my professors or the people in the class | Once class started, my fears dissipated | The class was enjoyable and the other students were friendly | I now love school and I do not get nervous anymore


In [18]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv
import random
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv("GEMMA_API_KEY"))

rng = random.Random(42)

with open("data/processed/train_processed.csv", 'r') as f:
    datareader = csv.reader(f)
    for row in datareader:
        original_story = row[1:2]
        shuffled_story = row[2]
    print("Original story:", " ".join(original_story))
    print("Shuffled story:", shuffled_story)

model = genai.GenerativeModel("gemini-2.0-flash-lite-001")

prompt = f"Reorder these sentences chronologically and output only the story in the same format (separated by |):\n\n{shuffled_story}"

response = model.generate_content(prompt)
print(f"Reconstructed story: {response.text}")

Original story: Starting my first day of college was terrifying | I was not sure of my professors or the people in the class | Once class started, my fears dissipated | The class was enjoyable and the other students were friendly | I now love school and I do not get nervous anymore
Shuffled story: Starting my first day of college was terrifying | Once class started, my fears dissipated | I now love school and I do not get nervous anymore | I was not sure of my professors or the people in the class | The class was enjoyable and the other students were friendly
Reconstructed story: Starting my first day of college was terrifying | I was not sure of my professors or the people in the class | Once class started, my fears dissipated | The class was enjoyable and the other students were friendly | I now love school and I do not get nervous anymore



In [23]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv
import random

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

rng = random.Random(42)

with open("data/processed/train_processed.csv", 'r') as f:
    datareader = csv.reader(f)
    mismatch_count = [0, 0, 0, 0, 0]
    for i, row in enumerate(datareader):
        if i == 0:
            continue
        if i >= 20:
            break
        original_story = row[1:2]
        shuffled_story = row[2]
        response = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "user", "content": f"Reorder these sentences chronologically keeping the same format: {shuffled_story}"}
            ]
        )
        reordered_story = response.choices[0].message.content
        original_story_arr = original_story[0].split('|')
        reordered_story_arr = reordered_story.split('|')
        for idx, sentence in enumerate(original_story_arr):
            if sentence != reordered_story_arr[idx]:
                mismatch_count[idx] += 1
    print(mismatch_count)

[2, 3, 1, 3, 3]


In [24]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv
import random
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv("GEMMA_API_KEY"))

rng = random.Random(42)
model = genai.GenerativeModel("gemini-2.0-flash-lite-001")

with open("data/processed/train_processed.csv", 'r') as f:
    datareader = csv.reader(f)
    mismatch_count = [0, 0, 0, 0, 0]
    for i, row in enumerate(datareader):
        if i == 0:
            continue
        if i >= 20:
            break
        original_story = row[1:2]
        shuffled_story = row[2]

        prompt = f"Reorder these sentences chronologically and output only the story in the same format (separated by |):\n\n{shuffled_story}"
        response = model.generate_content(prompt)
        
        reordered_story = response.text
        original_story_arr = original_story[0].split('|')
        reordered_story_arr = reordered_story.split('|')
        for idx, sentence in enumerate(original_story_arr):
            if sentence != reordered_story_arr[idx]:
                mismatch_count[idx] += 1
    print(mismatch_count)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).